In [ ]:
import random
from sklearn.metrics import accuracy_score

# Função para exibir o tabuleiro do jogo da velha de forma visual
def exibir_tabuleiro(tabuleiro):
    simbolos = {1: 'X', -1: 'O', 0: ' '}
    print("\n")
    print(f"{simbolos[tabuleiro[0]]} | {simbolos[tabuleiro[1]]} | {simbolos[tabuleiro[2]]}")
    print("--|---|--")
    print(f"{simbolos[tabuleiro[3]]} | {simbolos[tabuleiro[4]]} | {simbolos[tabuleiro[5]]}")
    print("--|---|--")
    print(f"{simbolos[tabuleiro[6]]} | {simbolos[tabuleiro[7]]} | {simbolos[tabuleiro[8]]}\n")

# Função para capturar a jogada do jogador humano
def jogada_humana(tabuleiro):
    while True:
        posicao = input("Escolha a posição (1-9) para jogar: ")
        if posicao.isdigit() and 1 <= int(posicao) <= 9 and tabuleiro[int(posicao) - 1] == 0:
            return int(posicao) - 1
        else:
            print("Posição inválida. Tente novamente.")

# Função para a jogada da IA, escolhendo aleatoriamente uma posição disponível
def jogada_ia(tabuleiro):
    movimentos_possiveis = [i for i, val in enumerate(tabuleiro) if val == 0]
    return random.choice(movimentos_possiveis)

# Função que usa o modelo treinado para verificar o estado do jogo
def verificar_estado(tabuleiro, modelo):
    estado = modelo.predict([tabuleiro])
    if estado == 1:
        return "Jogador X venceu!"
    elif estado == -1:
        return "Jogador O venceu!"
    elif 0 not in tabuleiro:
        return "Empate!"
    else:
        return "Tem jogo!"

# Função para registrar o estado real do jogo
def estado_real(tabuleiro):
    vitoria_X = [1, 1, 1]
    vitoria_O = [-1, -1, -1]
    
    combinacoes_vitoria = [
        tabuleiro[0:3], tabuleiro[3:6], tabuleiro[6:9],  # linhas
        tabuleiro[0::3], tabuleiro[1::3], tabuleiro[2::3],  # colunas
        [tabuleiro[0], tabuleiro[4], tabuleiro[8]],  # diagonal principal
        [tabuleiro[2], tabuleiro[4], tabuleiro[6]]  # diagonal secundária
    ]
    
    if vitoria_X in combinacoes_vitoria:
        return 1  # vitória do X
    elif vitoria_O in combinacoes_vitoria:
        return -1  # vitória do O
    elif 0 not in tabuleiro:
        return 0  # empate
    else:
        return None  # jogo ainda em andamento

# Função principal que executa o jogo da velha
def jogar(modelo):
    tabuleiro = [0] * 9  # inicia um tabuleiro vazio
    exibir_tabuleiro(tabuleiro)
    
    estados_previstos = []
    estados_reais = []
    
    while True:
        # turno do jogador humano (X)
        print("Sua vez! (Jogador X)")
        posicao = jogada_humana(tabuleiro)
        tabuleiro[posicao] = 1
        exibir_tabuleiro(tabuleiro)
        
        # verifica o estado real do jogo
        estado_real_atual = estado_real(tabuleiro)
        if estado_real_atual is not None:
            estados_reais.append(estado_real_atual)
        
        # verifica o estado do jogo com o modelo treinado
        estado_previsto = verificar_estado(tabuleiro, modelo)
        print(estado_previsto)
        
        # registrar o estado previsto para comparação
        estados_previstos.append(1 if estado_previsto == "Jogador X venceu!" else 0)
        
        if estado_previsto != "Tem jogo!":
            break
        
        # turno da IA (O)
        print("Vez da IA (Jogador O)...")
        posicao_ia = jogada_ia(tabuleiro)
        tabuleiro[posicao_ia] = -1
        exibir_tabuleiro(tabuleiro)
        
        # verifica o estado real do jogo após a jogada da IA
        estado_real_atual = estado_real(tabuleiro)
        if estado_real_atual is not None:
            estados_reais.append(estado_real_atual)
        
        # verifica o estado do jogo com o modelo treinado após a jogada da IA
        estado_previsto = verificar_estado(tabuleiro, modelo)
        print(estado_previsto)
        
        # registrar o estado previsto
        estados_previstos.append(-1 if estado_previsto == "Jogador O venceu!" else 0)
        
        if estado_previsto != "Tem jogo!":
            break

    # calcular e exibir a acurácia do modelo
    if len(estados_reais) > 0:
        acuracia = accuracy_score(estados_reais, estados_previstos)
        print(f"Acurácia da IA durante o jogo: {acuracia * 100:.2f}%")

# Exemplo de uso: rodar o jogo com um modelo (ex: k-NN treinado)
# Substitua `modelo_knn` pelo modelo treinado que deseja utilizar
# Exemplo de execução:
# jogar(modelo_knn)
